In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys
import os

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

In [10]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'name': 'test',
 'products':     description       stock_code
 0   Brazil nuts           363477
 1  GB chocolate           260803
 2          fake  fake_stock_code
 3        cheese           281739}

In [11]:
# Get and parse a product

code = w['products']['stock_code'].iat[0]
response = cd.get_product(code)
cd.parse_product(response)

OrderedDict([('stock_code', '363477'),
             ('name', 'Ceres Organics Brazil Nuts 250g'),
             ('description', None),
             ('size', '250g'),
             ('sale_price', 9.5),
             ('price', 11.29),
             ('discount_percentage', 15.854738706820193),
             ('unit_price', '$3.80/100G'),
             ('datetime', '2017-12-06T18:27:21')])

In [12]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=False)
f

CPU times: user 380 ms, sys: 8 ms, total: 388 ms
Wall time: 1.28 s


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.854739,$3.80/100G,2017-12-06 18:27:24
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,NaN,3.99,NaN,$3.99/100G,2017-12-06 18:27:24
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-12-06 18:27:25
3,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,7.3,11.00,33.636364,$14.60/1KG,2017-12-06 18:27:25


In [13]:
# Get many products asynchronously and parse

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=True)
f

CPU times: user 316 ms, sys: 4 ms, total: 320 ms
Wall time: 682 ms


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.854739,$3.80/100G,2017-12-06 18:27:28
1,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,NaN,3.99,NaN,$3.99/100G,2017-12-06 18:27:28
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-12-06 18:27:28
3,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,7.3,11.00,33.636364,$14.60/1KG,2017-12-06 18:27:28


In [14]:
# Filter sales

cd.filter_sales(f)

,name,sale_price,price,discount_percentage
3,Mainland Cheese Block Organic Cheddar 500g,7.3,11.00,33.636364
0,Ceres Organics Brazil Nuts 250g,9.5,11.29,15.854739


In [16]:
# Email sales

domain = os.environ['MAILGUN_DOMAIN'] # replace with your Mailgun domain
key = os.environ['MAILGUN_KEY'] # replace with your Mailgun API key
cd.email(f, ['brainbummer@mailinator.com'], domain, key)

<Response [200]>

In [30]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.yaml')

,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
2,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,7.3,11.00,33.6,$14.60/1KG,2017-12-06 19:23:27
1,363477,Ceres Organics Brazil Nuts 250g,None,250g,9.5,11.29,15.9,$3.80/100G,2017-12-06 19:23:27
0,260803,Green & Blacks Chocolate Block Organic Dark Ch...,None,100g,NaN,3.99,NaN,$3.99/100G,2017-12-06 19:23:27
3,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2017-12-06 19:23:27


In [27]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', OUT_DIR/'products.csv', domain, key)
%less {OUT_DIR/'products.csv'}